In [8]:
import sys
import os

# 添加项目根目录到路径
project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.insert(0, project_root)

from src.pipeline.tool_library_builder import ToolLibraryBuilder
from src.util.llm_client.get_llm_client import get_llm_client

In [9]:
# ========== 配置参数 ==========

# 问题类型: 'tsp', 'jssp', 'cvrp', 'psp'等
problem = "psp"

# 是否启用冒烟测试
smoke_test = False

# LLM配置文件路径
llm_config_file = os.path.join("output", "llm_config", "azure_gpt_4o.json")

# 最大重试次数
max_try_times = 5

print(f"问题类型: {problem}")
print(f"冒烟测试: {'启用' if smoke_test else '禁用'}")
print(f"LLM配置: {llm_config_file}")

问题类型: psp
冒烟测试: 禁用
LLM配置: output\llm_config\azure_gpt_4o.json


In [10]:
# ========== 检查依赖 ==========

print("检查必要的依赖文件...\n")

# 检查状态空间
state_file = os.path.join("output", problem, "generate_problem_state", "problem_state.py")
if os.path.exists(state_file):
    print("✅ State Space (S) 已生成")
else:
    print("⚠️  State Space (S) 未找到")
    print(f"   建议先运行: generate_problem_state.ipynb")

# 检查动作空间
action_file = os.path.join("output", problem, "generate_action_space", "action_space.py")
comp_file = os.path.join("src", "problems", problem, "components.py")
if os.path.exists(action_file) or os.path.exists(comp_file):
    print("✅ Action Space (A) 已生成")
else:
    print("⚠️  Action Space (A) 未找到")
    print(f"   建议先运行: generate_action_space.ipynb")

print("\n提示: 即使没有S和A,也可以生成工具库,但质量可能较低")

检查必要的依赖文件...

✅ State Space (S) 已生成
✅ Action Space (A) 已生成

提示: 即使没有S和A,也可以生成工具库,但质量可能较低


In [11]:
# ========== 初始化 ==========

# 设置目录
prompt_dir = os.path.join("src", "problems", "base", "prompt")
output_dir = os.path.join("output", problem, "generate_tool_library")

# 创建输出目录
os.makedirs(output_dir, exist_ok=True)

# 创建LLM客户端
print("初始化LLM客户端...")
llm_client = get_llm_client(llm_config_file, prompt_dir, output_dir)

# 创建Tool Library Builder
print("创建Tool Library Builder...")
tool_builder = ToolLibraryBuilder(llm_client=llm_client, problem=problem)

print("✅ 初始化完成!")

初始化LLM客户端...
创建Tool Library Builder...
✅ 初始化完成!


In [ ]:
# ========== 生成工具库 ==========

print("\n" + "="*60)
print("开始生成工具库...")
print("="*60 + "\n")

result_file = tool_builder.generate_tool_library(
    smoke_test=smoke_test,
    max_try_times=max_try_times
)

if result_file:
    print(f"\n✅ 工具库生成成功!")
    print(f"📁 文件位置: {result_file}")
    print(f"📄 文档位置: {result_file.replace('.py', '_documentation.txt')}")
else:
    print(f"\n❌ 工具库生成失败")
    print(f"请检查输出目录的日志文件: {output_dir}")


开始生成工具库...

开始生成工具库(Tool Library)

[步骤 1/4] 分析问题并确定需要的工具类别...
Chat dumped to output\psp\generate_tool_library\tool_categories.txt
  识别到 4 个工具类别:
    1. Tugboat_Combination_Optimization
    2. Service_Sequence_Coordination
    3. Resource_Conflict_Resolution
    4. Assignment_Impact_Evaluation

[步骤 2/4] 生成各类别的工具函数...
  [1/4] 生成 Tugboat_Combination_Optimization 相关工具...


In [ ]:
# ========== 查看生成结果 ==========

if result_file and os.path.exists(result_file):
    print("\n" + "="*60)
    print("生成的工具库代码:")
    print("="*60 + "\n")
    
    with open(result_file, 'r', encoding='utf-8') as f:
        code = f.read()
    
    # 显示前150行
    lines = code.split('\n')
    preview_lines = min(150, len(lines))
    
    print('\n'.join(lines[:preview_lines]))
    
    if len(lines) > preview_lines:
        print(f"\n... (共 {len(lines)} 行，显示前 {preview_lines} 行)")
    
    print("\n" + "="*60)
    print("工具库统计:")
    print("="*60)
    print(f"总行数: {len(lines)}")
    
    # 统计工具函数数量
    import re
    functions = re.findall(r'^def ([a-zA-Z_][a-zA-Z0-9_]*)\(', code, re.MULTILINE)
    print(f"工具函数: {len(functions)} 个")
    
    print("\n函数列表:")
    for i, func in enumerate(functions, 1):
        print(f"  {i}. {func}()")
else:
    print("未找到生成的文件")

In [ ]:
# ========== 查看工具文档 ==========

if result_file:
    doc_file = result_file.replace('.py', '_documentation.txt')
    
    if os.path.exists(doc_file):
        print("\n" + "="*60)
        print("工具库文档:")
        print("="*60 + "\n")
        
        with open(doc_file, 'r', encoding='utf-8') as f:
            doc = f.read()
        
        # 显示前100行
        doc_lines = doc.split('\n')
        preview_lines = min(100, len(doc_lines))
        
        print('\n'.join(doc_lines[:preview_lines]))
        
        if len(doc_lines) > preview_lines:
            print(f"\n... (共 {len(doc_lines)} 行，显示前 {preview_lines} 行)")
    else:
        print("文档文件未生成")